In [1]:
import time, re

In [2]:
def lat_lon_head_to_url(lat, lon, heading):
    return "https://www.google.com/maps/?q={lat},{lon}&h={head}".format(lat=lat, lon=lon, head=heading)


pattern = r'\d*\.\d*h'
reg = re.compile(pattern)
# find between @ and first /
pattern2 = r'(@.*?/)'
reg2 = re.compile(pattern2)
def change_url_heading(heading):
    time.sleep(1)
    url = driver.current_url
    splitted = re.split(pattern, url, maxsplit=1)
    try:
        new_url = splitted[0] + str(heading) + 'h' + splitted[1]
    except IndexError:
        splitted = re.split(reg2, url, maxsplit=1)
        try:
            new_url = splitted[0] + splitted[1][:-1] + ',' + str(heading) + 'h/' + splitted[2]
        except IndexError:
            print(splitted)
            
    time.sleep(4)
    driver.get(new_url)

def selenium_to_url_sv(url, driver):
    driver.get(url)
    try:
        driver.find_element_by_css_selector(
            "button.section-hero-header-image-hero.widget-pane-fading.widget-pane-fade-in.section-hero-header-image-hero-clickable"
        ).click()
    except NoSuchElement:
        raise
    
def selenium_look_right_snap(heading, driver, path):
    right_heading = (heading + 90) % 360
    change_url_heading(right_heading)
    time.sleep(5)
    driver.save_screenshot(path)
    

def seleneium_look_left_snap(heading, driver, path):
    left_heading = (heading - 90) % 360
    change_url_heading(left_heading)
    time.sleep(5)
    driver.save_screenshot(path)
    time.sleep(1)
    
def snap_lat_long(driver, lat, lon, heading, folder):
    url = lat_lon_head_to_url(lat, lon, heading)
    try: 
        selenium_to_url_sv(url,  driver)
    except:
        with open(folder + 'error', 'w') as error_file:
            print('error at ', lat, lon)
            error_file.write('could not get street view')
    selenium_look_right_snap(heading, driver, folder + str(lat) + ',' + str(lon) + '_' + 'right.png')
 
    seleneium_look_left_snap(heading, driver, folder + str(lat) + ',' + str(lon) + '_' + 'left.png')
 

In [3]:
import geopandas as gpd
streets = gpd.read_file("../data/brighton/brighton_streets.shp")

In [4]:
from shapely.geometry import GeometryCollection
from copy import deepcopy
def list_explode(gdf):
    rows = []
    old_rows = []
    for i, row in gdf.iterrows():
        mline = row['cut_geometry']
        if type(mline) == GeometryCollection:
            mline = mline.geoms
        new_rows = [deepcopy(row) for i in range(len(row['cut_geometry']))]
        for n, inline in enumerate(mline):
            new_rows[n]['geometry'] = inline
            old_rows.append(i)
        rows.extend(new_rows)
    new_gdf = gpd.GeoDataFrame(rows).reset_index(drop=True)
    new_gdf.crs = "EPSG:26919"
    return new_gdf


In [5]:
# Thank you Sean Gilles https://gist.github.com/sgillies/465156#file_cut.py
from shapely.geometry import LineString, Point
from copy import deepcopy
def cut(line, distance):
    # Cuts a line in two at a distance from its starting point
    if distance <= 0.0 or distance >= line.length:
        return [LineString(line)]
    coords = list(line.coords)
    for i, p in enumerate(coords):
        pd = line.project(Point(p))
        if pd == distance:
            return [
                LineString(coords[:i+1]),
                LineString(coords[i:])]
        if pd > distance:
            cp = line.interpolate(distance)
            return [
                LineString(coords[:i] + [(cp.x, cp.y)]),
                LineString([(cp.x, cp.y)] + coords[i:])]
        

def recursive_cut(line, distance):
    if [line] == cut(line, distance):
        return [line]
    else:
        segment, rest = cut(line, distance)
        return [segment] + recursive_cut(rest, distance)


In [6]:
streets = streets.to_crs("EPSG:26919")

In [7]:
def mls_to_list_safe(geom):
    if type(geom) == LineString:
        return [geom]
    elif (type(geom) == list and len(geom) == 1):
        return geom
    else:
        try:
            return geom.geoms
        except AttributeError:
            #list_of_geoms = list(geom)
            #[g for geom in list_of_geoms for g in geom]
            return list(geom)

streets['cut_geometry'] = streets.geometry.map(mls_to_list_safe)
streets = list_explode(streets)
streets['cut_geometry'] = streets.geometry.map(lambda x : recursive_cut(x,12))
streets_split = list_explode(streets)

In [8]:
streets_split['length_m'] = streets_split.length
streets_split.drop(inplace=True, columns=['length'])

streets_split = streets_split.to_crs("EPSG:4326")
streets_split.drop(inplace=True, columns=['cut_geometry'])

In [9]:
def midpoint(line):
    p0 = line.coords[0]
    p1 = line.coords[-1]
    x0, y0 = p0
    x1, y1 = p1
    return ((x0 + x1)/2, (y0 + y1)/2)

import math
def heading(line):
    # true north 0, east is 90
    # so 90 - arctan
    p0 = line.coords[0]
    p1 = line.coords[-1]
    x0, y0 = p0
    x1, y1 = p1
    
    seg = (x1 - x0, y1 - y0)
    angle = math.atan(seg[1]/seg[0]) * 180 / math.pi
    return ((90 - angle) % 360)
    

In [10]:
# beta reduce or whatever
streets_split['midpoint'] = streets_split.geometry.map(lambda x : midpoint(x))
streets_split['heading'] = streets_split.geometry.map(lambda x : int(heading(x)))

In [11]:
len(streets_split[streets_split['length_m'] > 11.9])

16042

In [12]:
with open("/home/adam/gapikey", 'r') as gfile:
    GKEY = gfile.readline().strip()

def row_dict(row):
    if row.length_m < 11.9:
        pass
    lon = str(row.midpoint[0])
    lat = str(row.midpoint[1])
    heading = row.heading
    return {
        'size': '1800x600',
        'heading': heading,
        'pitch': '0',
        'location': str(lat) + ',' + str(lon),
    }

def gsv_urls(row):
    dic = row_dict(row)
    init = "https://maps.googleapis.com/maps/api/streetview?"
    size = "size=" + dic['size']
    location = "&location=" + dic['location']
    heading_left = "&heading=" + str((dic['heading'] - 90) % 360)
    heading_right = "&heading=" + str((dic['heading'] + 90) % 360)
    pitch = "&pitch=" + dic['pitch']
    key = "&key=" + GKEY
    
    url_left =''.join([init,size,location,heading_left,pitch,key])
    url_right =''.join([init,size,location,heading_right,pitch,key])
    
    return (url_left, url_right)

In [13]:
import os.path
import requests as req
#https://maps.googleapis.com/maps/api/streetview?
#size=600x300&location=46.414382,10.013988&heading=151.78&pitch=-0.76&key=YOUR_API_KEY&signature=YOUR_SIGNATURE
from tqdm.autonotebook import tqdm

for _, row in tqdm(list(streets_split.iterrows())[1000:]):
    url_left = gsv_urls(row)[0]
    url_right = gsv_urls(row)[1]
    dic = row_dict(row)
    path_left = "../data/streetphotos/" + dic['location'] + '_' + 'left.jpg'
    path_right = "../data/streetphotos/" + dic['location'] + '_' + 'right.jpg'
    
    if os.path.isfile(path_left) and os.path.isfile(path_right):
        continue
        
#    print('requesting')
    pic_left = req.get(url_left).content
    pic_right = req.get(url_right).content
    
    with open(path_left, 'wb') as file_left:
        #pic_left.raw.decode_content = True
        file_left.write(pic_left) 
    with open(path_right, 'wb') as file_right:
        #pic_right.raw.decode_content = True
        file_right.write(pic_right) 
#    print('saved')

/home/adam/.local/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [ ]:
len(streets_split)